In [1]:
import requests
import pandas as pd


In [ ]:
base_url = "https://api.enerpricedata.com"
api_key = ""           # Replace with your actual API key


In [3]:
headers = {
    "X-API-Key": api_key  # FastAPI expects the key in this custom header
}


In [4]:
response = requests.get(f"{base_url}/openapi.json")
for path in response.json().get("paths", {}):
    print(path)


/graph/api/v1/energy-futures/graph-data
/graph/api/v1/energy-futures/load-zones
/graph/api/v1/rec/graph-data
/graph/api/v1/rec/states
/graph/api/v1/utility-price/graph-data
/graph/api/v1/utility-price/states
/graph/api/v1/utility-price/utilities
/graph/api/v1/utility-price/rate-classes
/graph/api/v1/ancillary-uplift/graph-data
/graph/api/v1/ancillary-uplift/load-zones
/graph/api/v1/ancillary-uplift/capacity-zones
/users
/users/{user_id}
/login
/verify-otp
/me
/logout
/forgot-password/request
/forgot-password/verify
/change-password
/datasets/upload/utility-price
/datasets/download/utility-price
/datasets/download/utility-price/summary/csv
/datasets/download/utility-price/details/csv
/datasets/download/utility-price/csv
/datasets/download/utility-price/json
/datasets
/datasets/{dataset_id}
/datasets/{dataset_id}/utility-price/summary
/datasets/{dataset_id}/utility-price/details
/datasets/{dataset_id}/utility-price/details/{detail_id}/ptc-values
/datasets/upload/energy-futures
/datasets/

Default:

In [5]:
import requests

base_url = "https://api.enerpricedata.com"
endpoint = "/me"

response = requests.get(f"{base_url}{endpoint}", headers=headers)

if response.status_code == 200:
    print("User Info:", response.json())
else:
    print("Error:", response.status_code, response.text)


User Info: {'id': 'f78fdfbf-3b5a-4e9f-8b8e-b2595da404ca', 'name': 'santosh', 'email': 'santosh@azr-holdings.com', 'role': 'admin', 'is_active': True, 'created_at': '2025-08-10T06:41:16.748649', 'updated_at': '2025-09-24T19:56:40.966833', 'api_key_id': 'b426811d-59c0-47e6-98f6-a1b2b82fdfff'}


### Energy Futures:

EXCEL

In [6]:
endpoint = "/datasets/download/energy-futures"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "end_operating_date": "2025-08-28",          # Optional, for date range
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "block_types": "7x8,2x16",                   # Optional: e.g., 7x8,2x16,5x16, etc. e.g., "7x8","2x16","5x16" -- By Default ALL 
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_EnergyFutures_{control_area}_{end_op_date}.xlsx"


if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    # If an error occurs, print status and response text
    print(f"Error {response.status_code}: {response.text}")

File downloaded successfully and saved as 'EPD_EnergyFutures_ERCOT_2025-08-28.xlsx'


CSV:

In [ ]:
endpoint = "/datasets/download/energy-futures/csv"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "block_types": "",                           # Optional: e.g., 7x8,2x16,5x16, etc. e.g., "7x8","2x16","5x16" -- By Default ALL 
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
start_op_date = params["start_operating_date"]
filename = f"EPD_EnergyFutures_{control_area}_{start_op_date}.csv"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    # status and response text
    print(f"Error {response.status_code}: {response.text}")

File downloaded successfully and saved as 'EPD_EnergyFutures_ERCOT_2025-08-28.csv'


JSON:

### Understanding Pagination Parameters

### Skip and Limit Explained

The JSON endpoints support pagination using `skip` and `limit` parameters:

- **`skip`**: Number of records to skip before starting to return results (offset)
- **`limit`**: Maximum number of records to return (default: 100, max: 1000)

### Examples:
- `skip=0, limit=100` → Returns records 1-100 (first page)
- `skip=100, limit=100` → Returns records 101-200 (second page)
- `skip=200, limit=50` → Returns records 201-250
- `skip=500, limit=100` → Returns records 501-600

#### Formula:
**Page Number = (skip ÷ limit) + 1**

In [ ]:
## Pagination Example: Getting All Records

# Paginating through all Energy Futures records
import requests

endpoint = "/datasets/download/energy-futures/json"
all_records = []
skip = 0
limit = 100  # Get 100 records per request

while True:
    params = {
        "start_operating_date": "2025-08-25",
        "control_area": "ERCOT",
        "raw": True,  # Get raw JSON data
        "skip": skip,
        "limit": limit
    }
    
    response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        # Check if we got any records
        if not data or len(data) == 0:
            print(f" Pagination complete! Total records collected: {len(all_records)}")
            break
            
        all_records.extend(data)
        print(f"Page {skip//limit + 1}: Retrieved {len(data)} records (Total: {len(all_records)})")
        
        # If we got fewer records than requested, we've reached the end
        if len(data) < limit:
            print(f" Reached end of data! Total records: {len(all_records)}")
            break
            
        # Move to next page
        skip += limit
    else:
        print(f" Error: {response.status_code} - {response.text}")
        break

print(f"\n Final Result: Collected {len(all_records)} total records")


In [ ]:
## Energy Futures JSON Download (Enhanced with Raw Output)

import json

endpoint = "/datasets/download/energy-futures/json"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "block_types": "7x8,2x16",                   # Optional: e.g., 7x8,2x16,5x16, etc. e.g., "7x8","2x16","5x16" -- By Default ALL 
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01",                    # Optional: filter end
    "raw": False,                                # Optional: if True, return raw JSON instead of file
    "skip": 0,                                   # Optional: pagination skip
    "limit": 100                                 # Optional: pagination limit (max 1000)
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
start_op_date = params["start_operating_date"]
filename = f"EPD_EnergyFutures_{control_area}_{start_op_date}.json"

if response.status_code == 200:
    if params.get("raw"):
        # If raw=True, response is JSON data directly
        data = response.json()
        print(f" Raw JSON data received: {len(data)} records")
        print(f"Sample record: {data[0] if data else 'No data'}")
    else:
        # If raw=False or not specified, response is a file
        data = response.json()
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
        print(f" File downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")


### REC/RPS Downloads

The REC/RPS dataset provides Renewable Energy Certificate and Renewable Portfolio Standard data.


#### Excel

In [ ]:
## REC/RPS Excel Download

endpoint = "/datasets/download/rec-rps"

params = {
    "start_operating_date": "2025-08-01",        # Required
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_REC_RPS_{control_area}_{end_op_date}.xlsx"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f" File downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")


#### CSV

In [ ]:
## REC/RPS CSV Download

endpoint = "/datasets/download/rec-rps/csv"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
start_op_date = params["start_operating_date"]
filename = f"EPD_REC_RPS_{control_area}_{start_op_date}.csv"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f" File downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")


#### JSON

In [ ]:
## REC/RPS JSON Download

import json

endpoint = "/datasets/download/rec-rps/json"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01",                    # Optional: filter end
    "raw": False,                                # Optional: if True, return raw JSON instead of file
    "skip": 0,                                   # Optional: pagination skip
    "limit": 100                                 # Optional: pagination limit (max 1000)
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
start_op_date = params["start_operating_date"]
filename = f"EPD_REC_RPS_{control_area}_{start_op_date}.json"

if response.status_code == 200:
    if params.get("raw"):
        # If raw=True, response is JSON data directly
        data = response.json()
        print(f" Raw JSON data received: {len(data)} records")
        print(f"Sample record: {data[0] if data else 'No data'}")
    else:
        # If raw=False or not specified, response is a file
        data = response.json()
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
        print(f" File downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")


### Ancillary Uplift Downloads

The Ancillary Uplift dataset provides data on ancillary services and uplift charges in electricity markets.


#### Excel:

In [ ]:
## Ancillary Uplift Excel Download

endpoint = "/datasets/download/ancillary-uplift"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "end_operating_date": "2025-08-25",          # Optional, for date range
    "control_area": "PJM",                       # ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_AncillaryUplift_{control_area}_{end_op_date}.xlsx"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f" File downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")


#### CSV

In [ ]:
## Ancillary Uplift CSV Download

endpoint = "/datasets/download/ancillary-uplift/csv"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "control_area": "PJM",                       # ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01"                     # Optional: filter end
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
start_op_date = params["start_operating_date"]
filename = f"EPD_AncillaryUplift_{control_area}_{start_op_date}.csv"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f" File downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")


#### JSON

In [ ]:
## Ancillary Uplift JSON Download

import json

endpoint = "/datasets/download/ancillary-uplift/json"

params = {
    "start_operating_date": "2025-08-29",        # Required
    "control_area": "ERCOT",                     # ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01",                    # Optional: filter end
    "raw": False,                                # Optional: if True, return raw JSON instead of file
    "skip": 0,                                   # Optional: pagination skip
    "limit": 100                                 # Optional: pagination limit (max 1000)
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
start_op_date = params["start_operating_date"]
filename = f"EPD_REC_RPS_{control_area}_{start_op_date}.json"

if response.status_code == 200:
    if params.get("raw"):
        # If raw=True, response is JSON data directly
        data = response.json()
        print(f" Raw JSON data received: {len(data)} records")
        print(f"Sample record: {data[0] if data else 'No data'}")
    else:
        # If raw=False or not specified, response is a file
        data = response.json()
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
        print(f" File downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")


### Utility Price:

#### Excel

In [ ]:
endpoint = "/datasets/download/utility-price"

params = {
    "start_operating_date": "2025-08-15",        # Required in YYYY-MM-DD format
    "end_operating_date": "2025-08-15",          # Optional, for date range
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_UtilityPrice_{end_op_date}.xlsx"


if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    # If an error occurs, print status and response text
    print(f"Error {response.status_code}: {response.text}")

CSV:

In [ ]:
endpoint = "/datasets/download/utility-price/csv"

params = {
    "start_operating_date": "2025-08-15",        # Required in YYYY-MM-DD format
    #"end_operating_date": "2025-08-15",          # Optional, for date range
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

end_op_date = params.get("end_operating_date", params["start_operating_date"])
filename = f"EPD_UtilityPrice_{end_op_date}.zip"


if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"File downloaded successfully and saved as '{filename}'")
else:
    #status and response text
    print(f"Error {response.status_code}: {response.text}")

#### JSON


In [ ]:
import json

endpoint = "/datasets/download/utility-price/json"

params = {
    "start_operating_date": "2025-08-15",        # Required
    "raw": False,                                # Optional: if True, return raw JSON instead of file
    "skip": 0,                                   # Optional: pagination skip
    "limit": 100                                 # Optional: pagination limit (max 1000)
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

start_op_date = params["start_operating_date"]
filename = f"EPD_UtilityPrice_{start_op_date}.json"

if response.status_code == 200:
    if params.get("raw"):
        # If raw=True, response is JSON data directly
        data = response.json()
        print(f" Raw JSON data received: {len(data)} records")
        print(f"Sample record: {data[0] if data else 'No data'}")
    else:
        # If raw=False or not specified, response is a file
        data = response.json()
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
        print(f" File downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")

## Utility Price Summary CSV Download

endpoint = "/datasets/download/utility-price/summary/csv"

params = {
    "start_operating_date": "2025-08-15",        # Required in YYYY-MM-DD format
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

start_op_date = params["start_operating_date"]
filename = f"EPD_UtilityPrice_Summary_{start_op_date}.csv"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f" Summary CSV downloaded successfully and saved as '{filename}'")
else:
    print(f" Error {response.status_code}: {response.text}")

### Bulk Download APIs

Bulk download APIs allow you to download data for multiple operating dates in a single Excel file, with each date as a separate sheet.


In [ ]:
## Bulk Utility Price Download

endpoint = "/datasets/download/bulk/utility-price"

params = {
    "start_operating_date": "2025-08-01",        # Required
    "end_operating_date": "2025-08-31",          # Required for bulk download
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

start_date = params["start_operating_date"]
end_date = params["end_operating_date"]
filename = f"EPD_UtilityPrice_Bulk_{start_date}_to_{end_date}.xlsx"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f" Bulk file downloaded successfully and saved as '{filename}'")
    print(" Each operating date will be in a separate sheet with Summary and PTC Details")
else:
    print(f" Error {response.status_code}: {response.text}")


In [ ]:
## Bulk Energy Futures Download

endpoint = "/datasets/download/bulk/energy-futures"

params = {
    "start_operating_date": "2025-08-01",        # Required
    "end_operating_date": "2025-08-31",          # Required for bulk download
    "control_area": "ERCOT",                     # Optional: ERCOT, PJM, ISONE, etc.
    "block_types": "7x8,2x16",                   # Optional: filter by block types
    "start_date": "2025-08-01",                  # Optional: additional date filter
    "end_date": "2030-09-01"                     # Optional: additional date filter
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

start_date = params["start_operating_date"]
end_date = params["end_operating_date"]
control_area = params.get("control_area", "ALL")
filename = f"EPD_EnergyFutures_Bulk_{control_area}_{start_date}_to_{end_date}.xlsx"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"Bulk file downloaded successfully and saved as '{filename}'")
    print("Each operating date will be in a separate sheet")
else:
    print(f"Error {response.status_code}: {response.text}")


In [ ]:
## Bulk REC/RPS Download

endpoint = "/datasets/download/bulk/rec-rps"

params = {
    "start_operating_date": "2025-08-01",        # Required
    "end_operating_date": "2025-08-31",          # Required for bulk download
    "control_area": "ISONE",                     # Optional: ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: additional date filter
    "end_date": "2030-09-01"                     # Optional: additional date filter
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

start_date = params["start_operating_date"]
end_date = params["end_operating_date"]
control_area = params.get("control_area", "ALL")
filename = f"EPD_REC_RPS_Bulk_{control_area}_{start_date}_to_{end_date}.xlsx"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"Bulk file downloaded successfully and saved as '{filename}'")
    print("Each operating date will be in a separate sheet")
else:
    print(f"Error {response.status_code}: {response.text}")


In [ ]:
## Bulk Ancillary Uplift Download

endpoint = "/datasets/download/bulk/ancillary-uplift"

params = {
    "start_operating_date": "2025-08-01",        # Required
    "end_operating_date": "2025-08-31",          # Required for bulk download
    "control_area": "PJM",                       # Optional: ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: additional date filter
    "end_date": "2030-09-01"                     # Optional: additional date filter
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

start_date = params["start_operating_date"]
end_date = params["end_operating_date"]
control_area = params.get("control_area", "ALL")
filename = f"EPD_AncillaryUplift_Bulk_{control_area}_{start_date}_to_{end_date}.xlsx"

if response.status_code == 200:
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"Bulk file downloaded successfully and saved as '{filename}'")
    print("Each operating date will be in a separate sheet")
else:
    print(f"Error {response.status_code}: {response.text}")


In [ ]:
## Ancillary Uplift JSON Download

import json

endpoint = "/datasets/download/ancillary-uplift/json"

params = {
    "start_operating_date": "2025-08-25",        # Required
    "control_area": "PJM",                       # ERCOT, PJM, ISONE, etc.
    "start_date": "2025-08-01",                  # Optional: filter start
    "end_date": "2030-09-01",                    # Optional: filter end
    "raw": False,                                # Optional: if True, return raw JSON instead of file
    "skip": 0,                                   # Optional: pagination skip
    "limit": 100                                 # Optional: pagination limit (max 1000)
}

response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)

control_area = params["control_area"]
start_op_date = params["start_operating_date"]
filename = f"EPD_AncillaryUplift_{control_area}_{start_op_date}.json"

if response.status_code == 200:
    if params.get("raw"):
        # If raw=True, response is JSON data directly
        data = response.json()
        print(f"Raw JSON data received: {len(data)} records")
        print(f"Sample record: {data[0] if data else 'No data'}")
    else:
        # If raw=False or not specified, response is a file
        data = response.json()
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)
        print(f"File downloaded successfully and saved as '{filename}'")
else:
    print(f"Error {response.status_code}: {response.text}")


### Complete API Summary


## Dataset Types Available

#### 1. **Energy Futures** (`/datasets/download/energy-futures`)
- **Excel**: Single or bulk download with multiple sheets
- **CSV**: Single date download
- **JSON**: Single date with pagination support
- **Parameters**: `control_area`, `block_types`, date filters
- **Bulk**: `/datasets/download/bulk/energy-futures`

#### 2. **Utility Price** (`/datasets/download/utility-price`)
- **Excel**: Single or bulk download
- **CSV**: Combined zip file, or separate summary/details endpoints
- **JSON**: Single date with pagination support
- **Special Endpoints**: 
  - `/datasets/download/utility-price/summary/csv`
  - `/datasets/download/utility-price/details/csv`
- **Bulk**: `/datasets/download/bulk/utility-price`

#### 3. **REC/RPS** (`/datasets/download/rec-rps`)
- **Excel**: Single or bulk download with multiple sheets
- **CSV**: Single date download
- **JSON**: Single date with pagination support
- **Parameters**: `control_area`, date filters
- **Bulk**: `/datasets/download/bulk/rec-rps`

#### 4. **Ancillary Uplift** (`/datasets/download/ancillary-uplift`)
- **Excel**: Single or bulk download with multiple sheets
- **CSV**: Single date download
- **JSON**: Single date with pagination support
- **Parameters**: `control_area`, date filters
- **Bulk**: `/datasets/download/bulk/ancillary-uplift`

###  Common Parameters

#### Required Parameters
- `start_operating_date`: Date in YYYY-MM-DD format

#### Optional Parameters
- `end_operating_date`: For bulk downloads (date range)
- `control_area`: ERCOT, PJM, ISONE, etc.
- `start_date`/`end_date`: Additional date filters
- `raw`: Return JSON data directly (JSON endpoints only)
- `skip`/`limit`: Pagination (JSON endpoints only)
- `block_types`: Energy Futures specific filter

###  Authentication
All endpoints require API key authentication via `X-API-Key` header.

### Complete Coverage
This notebook now includes **ALL** download center APIs:
- 4 Dataset types × 3 formats = 12 standard endpoints
- 4 Bulk download endpoints  
- 2 Utility Price specific CSV endpoints
- Proper error handling and parameter documentation
- **Total: 18 API endpoints covered**
